In [1]:
# Jupyter notebook related
%reload_ext autoreload
%autoreload 2
%matplotlib widget

In [4]:
from dataset import get_train_val_locs

train_locs, val_locs = get_train_val_locs(n_val_locs=10000)

2024-11-06 01:07:45.437 | WARNING  | dataset:get_train_val_locs:285 - n_val_locs is greater than the maximum number of samples available. Setting n_val_locs to 7336


(7336, 28)


In [3]:
from dataset import EvoNetV1Dataset

/home/dzanaga/miniconda3/envs/eo/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/home/dzanaga/miniconda3/envs/eo/lib/python3.10/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any 

In [5]:
ds = EvoNetV1Dataset()

In [6]:
feats, feats_head, target = ds[0]

In [7]:
feats.shape, feats_head.shape, target.shape

(torch.Size([36, 128, 128]),
 torch.Size([9, 128, 128]),
 torch.Size([9, 128, 128]))

In [8]:
import json

with open("configs/lcm10-unet-base-v06_debug_mep.json", "r") as f:
    config = json.load(f)

In [9]:
from model import Net

In [10]:
net = Net(**config["model"])

Initializing Conv2d(16, 20, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), padding_mode=reflect)
Initializing Conv2d(16, 20, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), padding_mode=reflect)
Initializing Conv2d(16, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
Initializing Conv2d(16, 20, kernel_size=(1, 1), stride=(1, 1), padding_mode=reflect)
Initializing Linear(in_features=89, out_features=300, bias=True)
Initializing Linear(in_features=300, out_features=200, bias=True)
Initializing Linear(in_features=200, out_features=200, bias=True)
Initializing Linear(in_features=200, out_features=100, bias=True)
Initializing Linear(in_features=100, out_features=9, bias=True)


In [11]:
y = net(feats.unsqueeze(0), feats_head.unsqueeze(0))

In [12]:
from dataset import get_train_val_dataloaders

train_loader, val_loader, train_dataset, val_dataset = (
    get_train_val_dataloaders("configs/lcm10-unet-base-v06_debug_mep.json")
)

/home/dzanaga/miniconda3/envs/eo/lib/python3.10/site-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
batch = next(iter(train_loader))

/home/dzanaga/miniconda3/envs/eo/lib/python3.10/site-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


: 

In [ ]:
ybatch = net(batch[0], batch[1])

In [ ]:
import numpy as np

from evotrain.v2 import EvoTrainV2Dataset

evo_ds = EvoTrainV2Dataset()
v3_locs = evo_ds.locs
v3_locs = v3_locs[v3_locs.aux]  # discard locs without aux tifs. to check

group0 = v3_locs[v3_locs.group == 0]
train_locs0 = group0[
    (group0.lab_50 > 0.05) | (group0.lab_70 > 0.05) | (group0.lab_20 > 0.05)
].location_id.values

group1 = v3_locs[v3_locs.group == 1]
train_locs = group1.location_id.values
train_locs = np.concatenate([train_locs0, train_locs])


# group2 = v3_locs[v3_locs.group == 2].sample(n=n_val_locs, random_state=42)
# val_locs = group2.location_id.values

In [6]:
lab_cols = [col for col in v3_locs.columns if "lab" in col]

v3_locs[v3_locs.location_id.isin(train_locs)][lab_cols].mean(axis=0)

lab_10     0.216607
lab_20     0.188427
lab_30     0.245537
lab_40     0.097034
lab_50     0.040324
lab_60     0.069956
lab_70     0.006992
lab_80     0.076982
lab_90     0.032144
lab_95     0.007721
lab_100    0.018275
dtype: float64

In [ ]:
train_locs0 = group0[
    (group0.lab_50 > 0.05) | (group0.lab_70 > 0.05) | (group0.lab_20 > 0.72)
].location_id.values
train_locs0.size

8133

In [ ]:
group2 = v3_locs[v3_locs.group == 2]
lab_cols = [col for col in group2.columns if "lab" in col]

val_locs = []
for lab in lab_cols:
    if lab in ["lab_10", "lab_30"]:
        continue
    g = group2[group2[lab] > 0.30]
    val_locs += g.sample(min(1000, len(g))).location_id.values.tolist()


In [31]:
len(val_locs)

7581

In [1]:
from dataset import get_train_val_locs

train_locs, val_locs = get_train_val_locs(n_val_locs=10000)

/home/dzanaga/miniconda3/envs/eo/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/home/dzanaga/miniconda3/envs/eo/lib/python3.10/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any 

ValueError: Cannot take a larger sample than population when 'replace=False'